In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from tf_agents.environments import tf_py_environment
from tf_agents.environments.utils import validate_py_environment

from environments.PPO_environment import FitnessLandscapeEnvironment as FLEnv
from explorers.PPO_explorer import PPO_explorer

from utils.sequence_utils import generate_random_mutant, generate_random_sequences

## 1. Building oracles for landscapes

In [20]:
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor

In [31]:
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()

B1L14RNA1 loaded


In [32]:
landscape_constructor.loaded_landscapes

[{'B1L14RNA1': {'self_fold_max': False,
   'seq_size': 14,
   'conserved_start': False,
   'conserved_pattern': False,
   'targets': ['GAACGAGGCACAUUCCGGCUCGCCCGGCCCAUGUGAGCAUGGGCCGGACCCCGUCCGCGCGGGGCCCCCGCGCGGACGGGGGCGAGCCGGAAUGUGCCUC'],
   'starts': {'startRNAL14_0': 'AUGGGCCGGACCCC',
    'startRNAL14_1': 'GCCCCGCCGGAAUG',
    'startRNAL14_2': 'UCUUGGGGACUUUU',
    'startRNAL14_3': 'GGAUAACAAUUCAU',
    'startRNAL14_4': 'CCCAUGCGCGAUCA'}}}]

In [33]:
#take one landscape oracle from the generator
landscape_object=next(landscape_generator)

In [38]:
print(landscape_object['landscape_id'])
print(landscape_object['starting_seqs'])
landscape_oracle = landscape_object['landscape_oracle']
print(landscape_oracle.get_fitness('GGAUAACAAUUCAU'))
wt = landscape_object['starting_seqs']['startRNAL14_3']

B1L14RNA1
{'startRNAL14_0': 'AUGGGCCGGACCCC', 'startRNAL14_1': 'GCCCCGCCGGAAUG', 'startRNAL14_2': 'UCUUGGGGACUUUU', 'startRNAL14_3': 'GGAUAACAAUUCAU', 'startRNAL14_4': 'CCCAUGCGCGAUCA'}
0.14554841726711917


## 2. Building a model of the landscape

In [61]:
from utils.model_architectues import CNNa
from models.Noisy_models.Neural_network_models import NN_model

RNA_alphabet = "UCGA"
training_sequences = list(set([generate_random_mutant(wt, 0.2, RNA_alphabet) for i in range(0,500)]))
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

## 3. Exploring the landscape with a model

In [62]:
from explorers.base_explorer import Random_explorer

mutation_rate = 0.05
explorer = Random_explorer(mutation_rate,
                           batch_size=100,
                           alphabet=RNA_alphabet,
                           virtual_screen=10)
explorer.debug = True # don't save results
explorer.set_model(noisy_nn_model)

In [63]:
num_of_rounds = 5
explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 341, evals: 0, top: 0.41192946800707114
round: 1, cost: 441, evals: 100, top: 0.41192946800707114
round: 2, cost: 541, evals: 200, top: 0.4366452256115873
round: 3, cost: 641, evals: 300, top: 0.45037620787848964
round: 4, cost: 741, evals: 400, top: 0.4668533970746806


### PPO explorer

In [64]:
fle = FLEnv(alphabet=RNA_alphabet,
          starting_seq=wt,
          landscape=noisy_nn_model,
          max_num_steps=10)

validate_py_environment(fle, episodes=1)

tf_fle = tf_py_environment.TFPyEnvironment(fle)

In [65]:
agent = PPO_explorer.initialize_new_agent(tf_fle)

In [54]:
ppo_explorer = PPO_explorer(
    ppo_agent=agent,
    tf_env=tf_fle,
    alphabet=RNA_alphabet)
ppo_explorer.debug = True
ppo_explorer.set_model(noisy_nn_model)

In [ ]:
ppo_explorer.train_agent()

In [67]:
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

num_of_rounds = 5
ppo_explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 1384, evals: 3928, top: 0.9117372434741337
round: 1, cost: 1473, evals: 4017, top: 0.9117372434741337
round: 2, cost: 1568, evals: 4112, top: 0.9117372434741337
round: 3, cost: 1657, evals: 4201, top: 0.9117372434741337
round: 4, cost: 1750, evals: 4294, top: 0.9117372434741337
